In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [6]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [7]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

In [8]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

In [9]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

In [10]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [11]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

In [12]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

In [15]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

In [16]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...


I need to find a venue in San Francisco that meets the criteria for the Tech Innovation Conference.

Action: Search the internet
Action Input: {"search_query": "tech conference venues in San Francisco"} 


Search results: Title: Top 12 tech conferences in San Francisco for 2024 - TravelPerk
Link: https://www.travelperk.com/blog/top-tech-conferences-in-san-francisco/
Snippet: Running for over twenty years, Dreamforce -- Salesforce's flagship annual conference -- is widely known as one of the biggest tech conferences ...
---
Title: Discover Tech Conferences Events & Activities in San Francisco, CA
Link: https://www.eventbrite.com/d/ca--san-francisco/conferences/tech-conferences/
Snippet: Tech conferences events in San Francisco, CA ; Silicon Valley AI and Tech Investing Summit. Wed, May 22 • 12:00 PM. 2490 Middlefield Rd. From $20.00 ; Code for ...
---
Title: Tech Summit | The Home of Technology
Link: https://techsummit.tech/
Snippet: Embark on a global tech exploration with Tech Summit!



> Entering new CrewAgentExecutor chain...


> Entering new CrewAgentExecutor chain...
I need to effectively market the Tech Innovation Conference and engage with potential attendees to maximize event exposure and participation. 

Action:
Delegate work to co-worker
Action Input:
{
  "task": "Research top tech blogs for guest posting opportunities to promote the conference",
  "context": "I need to find ways to increase exposure for the Tech Innovation Conference and attract more attendees.",
  "coworker": "Venue Coordinator"
}

> Entering new CrewAgentExecutor chain...
I need to coordinate catering and equipment for the TECHSPO Silicon Valley 2022 event with 500 participants on 2024-09-15.

Action: Delegate work to co-worker
Action Input: {"task": "Coordinate catering for event on 2024-09-15 with 500 participants at Airport Marriott Waterfront, San Francisco, California", "context": "Need to ensure a variety of food options are available for all attendees"} 

I encountered an error wh

Exception in thread Thread-12 (_execute):
Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/crewai/task.py", line 192, in _execute
    result = agent.execute_task(
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/crewai/agent.py", line 236, in execute_task
    result = self.agent_executor.invoke(
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/langchain/chains/base.py", line 153, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/home/zeus/miniconda3/envs/cloudspace/lib/p